In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments,
                           ema_logging, MultiprocessingEvaluator)

from ema_workbench.connectors.netlogo import NetLogoModel

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis.plotting import lines, Density


def PredPrey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, final_time=365, reps=1):

    #Initial values
    predators, prey, sim_time = [np.zeros((reps, int(final_time/dt)+1)) for _ in range(3)]
    
    for r in range(reps):
        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

        #Calculate the time series
        for t in range(0, sim_time.shape[1]-1):

            dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
            dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

            prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
            predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
            sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}


C:\Users\Georgette\anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:17: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)


In [2]:
'''

This example is a proof of principle for how NetLogo models can be
controlled using pyNetLogo and the ema_workbench. Note that this
example uses the NetLogo 6 version of the predator prey model that
comes with NetLogo. If you are using NetLogo 5, replace the model file
with the one that comes with NetLogo.

'''
from __future__ import unicode_literals, absolute_import


from ema_workbench import (RealParameter, ema_logging, TimeSeriesOutcome, MultiprocessingEvaluator)


if __name__ == '__main__':
    # turn on logging
    ema_logging.log_to_stderr(ema_logging.INFO)

    model = NetLogoModel('predprey',
                         wd="./models",
                         model_file="NetlPredPrey.nlogo")
    model.run_length = 365
    model.replications = 4 
    #is this right or should this be 4? or just 1?

model.uncertainties = [RealParameter("prey_births_rate", 0.015, 0.035),  # we can refer to a cell in the normal way # we can also use named cells
                           RealParameter("predation_rate", 0.0005, 0.003),
                           RealParameter("predator_efficiency", 0.001, 0.004),
                           RealParameter("predator_loss_rate", 0.04, 0.08)]

model.outcomes = [TimeSeriesOutcome('prey'),
                      TimeSeriesOutcome('predator')]

    # perform experiments
n = 50

with MultiprocessingEvaluator(model, n_processes=2, maxtasksperchild=4) as evaluator:
        results = evaluator.perform_experiments(n)


[MainProcess/INFO] pool started
[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\Georgette\anaconda3\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 91, in run_experiment
    self.cleanup()
  File "C:\Users\Georgette\anaconda3\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 45, in cleanup
    msi.cleanup()
  File "C:\Users\Georgette\anaconda3\lib\site-packages\ema_workbench\connectors\netlogo.py", line 258, in cleanup
    jpype.shutdownJVM()
  File "C:\Users\Georgette\anaconda3\lib\site-packages\jpype\_core.py", line 314, in shutdownJVM
    _jpype.shutdown()
RuntimeError: Attempt to shutdown without a live JVM


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Georgette\anaconda3\lib\multiprocessing\pool.py", line 121, in worker
    res

KeyboardInterrupt: 